In [1]:
#In linear regression model,each target variable is estimated to be a weighted 
#sum of the input variables,offset by some constant known as bias

#yield=weight1* parameter1+weight1*parameter2+weightn*parametern + b(bias)

In [66]:
import numpy as np
import torch

In [67]:
#Input(temp,rainfall,humidity)

inputs=np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype="float32")


In [68]:
#Targets(apples,oranges)

targets=np.array([[56,70],
                 [81,101],
                 [119,133],
                 [22,37],
                 [103,119]],dtype="float32")

In [69]:
#convert inputs and targets to tensors
inputs=torch.from_numpy(inputs)
targets=torch.from_numpy(targets)
print(inputs)
print(targets)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [70]:
#weights and biases
w=torch.randn(2,3,requires_grad=True)
b=torch.randn(2,requires_grad=True)
print(w)
print(b)

#torch.randn creates a tensors of given dimensions with mean of 0 and standard deviation 
#of 1 (normal distribution)

tensor([[-1.7509,  0.4957, -0.1962],
        [ 0.7992,  0.0389,  0.8197]], requires_grad=True)
tensor([-1.6893, -0.3677], requires_grad=True)


In [71]:
def model(x):
    return x @ w.t() +b


In [72]:
#w.t() returns the transpose of the tensor 
#@ is for matrix multiplication
#python makes first column of b repeat n times where n is the number of rows in the result of matrix multiplication

In [73]:
#Generate predictions
preds=model(inputs)
print(preds)

tensor([[-104.7326,   95.8279],
        [-129.9599,  128.2440],
        [ -98.9789,  121.9200],
        [-166.2278,  113.1523],
        [ -88.6516,  115.8910]], grad_fn=<AddBackward0>)


In [76]:
#compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [77]:
diff=preds-targets
torch.sum(diff*diff)/diff.numel()   #to find mean squared error

#torch.sum(tensor_name) it adds all the elements in the tensor
#matrix_name.numel() gives the number of elements in the tensor


tensor(19735.4785, grad_fn=<DivBackward0>)

In [79]:
#compute Loss function (mean squared error) 

def mse(t1,t2):
    diff=t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [80]:
loss=mse(preds,targets)
print(loss)


tensor(19735.4785, grad_fn=<DivBackward0>)


In [81]:
#compute gradients(derivatives)
#loss is a quadratic function of weigths and biases
#inputs and targets are not changing only the weights and biases are changing
loss.backward()

In [83]:
print(w)
print(w.grad) #returns the derivatives

tensor([[-1.7509,  0.4957, -0.1962],
        [ 0.7992,  0.0389,  0.8197]], requires_grad=True)
tensor([[-16463.6367, -17007.0137, -10687.1494],
        [  2190.7378,   1123.8601,    962.3153]])


In [86]:
#objective to find set of weights where loss is the lowest
#the derivative of loss curve(loss function) represents the slope of loss function with respect to the weights and biases (the rate of change of loss)
#move in opposite direction of sign of slope
#derivative positive decrease the weights slightly
#derivative negative increase the weights slightly

In [88]:
w.grad.zero_()   #after use make the grad value back to zero as pytorch dust not replace it will keep on adding to the gradients when backward is called again
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [90]:
#revision repeat of steps
#generate predictions
preds=model(inputs)
print(preds)

tensor([[-104.7326,   95.8279],
        [-129.9599,  128.2440],
        [ -98.9789,  121.9200],
        [-166.2278,  113.1523],
        [ -88.6516,  115.8910]], grad_fn=<AddBackward0>)


In [91]:
#calculate the loss
loss=mse(preds,targets)
print(loss)

tensor(19735.4785, grad_fn=<DivBackward0>)


In [92]:
#compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-16463.6367, -17007.0137, -10687.1494],
        [  2190.7378,   1123.8601,    962.3153]])
tensor([-193.9101,   23.0070])


In [96]:
#Adjust weights and reset gradients  very very important
#torch.no_grad to indicate to pytorch that we shouldn't track,calculate or modify gradients while updating the weights and biases
#1e-5 this means 1 *10 to the power -5 
#subtracting the gradient from the weights decreases the weight but sometimes the gradient values are large so 
#we subtract the weights by a small portion the gradients
#negative gradients will automatically add as negative of negative is positive thereby increasing the value

#as
#move in opposite direction of sign of slope
#derivative positive decrease the weights slightly
#derivative negative increase the weights slightly

with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()


In [97]:
print(w)
print(b)

tensor([[-1.5863,  0.6657, -0.0893],
        [ 0.7773,  0.0277,  0.8100]], requires_grad=True)
tensor([-1.6874, -0.3679], requires_grad=True)


In [98]:
#calculate loss
preds=model(inputs)
loss=mse(preds,targets)
print(loss)


tensor(13529.6592, grad_fn=<DivBackward0>)


In [99]:
#putting everything in a loop
#every run of this loop is called epoch in machine learning
for i in range(100):
    preds=model(inputs)
    loss=mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w.grad*1e-5
        b-=b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()

        

In [100]:
#calculate loss
preds=model(inputs)
loss=mse(preds,targets)
print(loss)


tensor(207.7222, grad_fn=<DivBackward0>)


In [101]:
#predictions
preds

tensor([[ 54.1546,  75.7929],
        [ 77.6295, 104.6361],
        [133.9382, 115.1215],
        [  4.9585,  69.2762],
        [103.0351, 107.1375]], grad_fn=<AddBackward0>)

In [102]:
#targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])